# Final Training & Evaluation
* In this notebook we are going to train our final model in multiple iterations
* We are going to utilize what we experimented with in previous notebooks and use the learnings here.
* Below are the general steps to train final model,
    * Step 1: Create a custom model class & optimized loss function calculation to train the model. This loss function will calcualate the loss only once instead of 4 times in previous version
    * Step 2: Train a model to detect 2 objects on the canvas. 
    * Step 3: We'll than select the best model and train it to detect 3 objects on the canvas, then 4 objects and so on. 


In [30]:
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import mnist
import matplotlib.pyplot as plt
import matplotlib.patches as patches



## validate tensorflow 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Constants

In [31]:
data_dir = Path("..","data")
models_dir = Path("..","models")

In [32]:
import os
import sys
# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import graph_compatible_data_generator,object_detection_model,training_utils

## logic to auto reload scripts without restarting the kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize Pipeline

In [33]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X_tensor = tf.convert_to_tensor(x_train, dtype=tf.float32)
# X_tensor = tf.reshape(X_tensor, shape=(-1, 28, 28, 1))
y_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)

batch_size = 32
raw_dataset = tf.data.Dataset.from_tensor_slices((X_tensor, y_tensor))

# create a generator for 2 digits
data_gen_2_digits = graph_compatible_data_generator.create_data_generator(2)

processed_dataset_2 = raw_dataset.map(
    data_gen_2_digits).batch(batch_size=batch_size).prefetch(tf.data.AUTOTUNE)

## Initialize Model

In [34]:
inputs = tf.keras.Input(shape=(100,100,1),batch_size=batch_size ,name="input_layer")

x = tf.keras.layers.Rescaling(scale=1./255, name="rescaling")(inputs)

x = tf.keras.layers.Conv2D(filters=8, kernel_size=5, padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=8, kernel_size=5, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

outputs = tf.keras.layers.Conv2D(filters=45, kernel_size=1, padding='same', activation='linear')(x)

# Define the final model by specifying its inputs and outputs
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (32, 100, 100, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (32, 100, 100, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (32, 100, 100, 8)      │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (32, 100, 100, 8)      │         1,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (32, 50, 50, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (32, 50, 50, 8)        │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (32, 50, 50, 8)        │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (32, 25, 25, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (32, 25, 25, 16)       │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (32, 25, 25, 16)       │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (32, 12, 12, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (32, 12, 12, 32)       │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (32, 12, 12, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (32, 6, 6, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (32, 6, 6, 45)         │         1,485 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,845 (85.33 KB)

 Trainable params: 21,845 (85.33 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Define the callbacks
checkpoint_filepath = '../models/experiment_0_digits_2_{epoch:02d}_{loss:.2f}.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True,
    save_freq="epoch",
    verbose=1,
    )

In [36]:
custom_model = object_detection_model.ObjectDetectionModel(model)


custom_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,clipnorm=1.0))

custom_model.build(input_shape=(None, 100, 100, 1))

In [37]:
## Fit the model
epochs=20

history = custom_model.fit(
  processed_dataset_2,
  epochs=epochs,
  callbacks=[model_checkpoint_callback]
)


Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - bbox_loss: 0.1386 - class_loss: 2.1142 - loss: 2.5836 - obj_less_loss: 0.3043 - obj_loss: 0.1649
Epoch 1: loss improved from inf to 2.11401, saving model to ../models/experiment_0_digits_2_01_2.11.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 251s 126ms/step - bbox_loss: 0.1386 - class_loss: 2.1140 - loss: 2.5834 - obj_less_loss: 0.3043 - obj_loss: 0.1649
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - bbox_loss: 0.1768 - class_loss: 1.2740 - loss: 1.4163 - obj_less_loss: 0.0951 - obj_loss: 0.0469
Epoch 2: loss improved from 2.11401 to 1.31504, saving model to ../models/experiment_0_digits_2_02_1.32.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 242s 126ms/step - bbox_loss: 0.1768 - class_loss: 1.2740 - loss: 1.4162 - obj_less_loss: 0.0951 - obj_loss: 0.0469
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - bbox_loss: 0.1768 - class_loss: 0.9340 - loss: 1.0657 - obj_less_loss: 0.0884 - obj_loss: 0.0431
Epoch 3: loss improved from

* The model performance is descent, but the bounding boxes are still not pefect. 